# Pergunta a ser respondida:

- Classificar se um indivíduo sobreviveu ou morreu ao afundamento do Titanic.

  - Temos uma pergunta de classificação, pois estamos fazendo a predição de um rótulo (label) acerca da sobrevivência: um passageiro sobreviveu ou morreu.

## Dicionários dos atributos:

- pclass: classe do passageiro (1=primeiroa, 2=segunda, 3=terceira);
- survived: sobreviveu (0=não, 1=sim);
- name: nome;
- sex: sexo;
- age: idade;
- sibsp: número de irmãos/esposa(o) a bordo;
- parch: número de pais/filhos a bordo;
- ticket: número da passagem;
- fare: preço da passagem;
- cabin: cabine;
- embarked: local em que o passageiro embarcou (C=Cherbourg, Q=Queenstown, S=Southampton);
- boat: bote salva-vidas;
- body: número de identificação do corpo;
- home.dest: lar/destino;

## Refatoração do código

### Imports necessários

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
from sklearn import (
    ensemble,
    preprocessing,
    tree
)
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn import model_selection
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold
)
from yellowbrick.model_selection import (
    LearningCurve
)
from sklearn.experimental import (enable_iterative_imputer)
from sklearn import impute

### Extração dos dados e formação do dataframe

In [2]:
df = pd.read_csv("titanic3.csv")

## Funções:

### Limpeza dos dados

- exclusão dos campos desnecessários;
- transformação das variáveis dummies;

In [3]:
def limpeza_dados(df):
  df = df.drop(
            columns=[
                      "name",
                      "ticket",
                      "home.dest",
                      "boat",
                      "body",
                      "cabin"
  ]).pipe(pd.get_dummies, drop_first=True) # O uso do pipe permite que essas operações sejam encadeadas de forma mais limpa e eficiente

  return df

### Divisão de dados para treino e teste e imputando dados ausentes

In [4]:
def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
  y = df[y_col]
  X = df.drop(columns=y_col)
  X_train, X_test, y_train, y_test = model_selection.train_test_split(
                                                                      X, y, test_size=size, random_state=42
                                                                    )
  cols = X.columns
  num_cols = [
                "pclass",
                "age",
                "sibsp",
                "parch",
                "fare"
              ]

  fi = impute.IterativeImputer()
  X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
  X_test.loc[:, num_cols] = fi.transform(X_test[num_cols])

  if std_cols:
    std = preprocessing.StandardScaler()
    X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
    X_test.loc[:, std_cols] = std.transform(X_test[std_cols])

    return X_train, X_test, y_train, y_test

#.loc[:, num_cols] - pega todas as linhas (:) e as coliunas indicadas na variável num_cols

## Chamando as funções

In [5]:
ti_df = limpeza_dados(df)
std_cols = "pclass,age,sibsp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)

## Modelo - teste 1

In [6]:
from sklearn.dummy import DummyClassifier

bm = DummyClassifier()

bm.fit(X_train, y_train)
bm.score(X_test, y_test) # precisão

0.5699745547073791

### Vendo a métrica da precisão

In [7]:
from sklearn import metrics

metrics.precision_score(y_test, bm.predict(X_test))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0